## 6.15. Balanced Binary Search Trees

__AVL Tree__는 Node가 계속 변경되어도 Tree가 계속 Balanced한 상태로 유지되도록 한다. AVL Tree는 __Balance Factor__를 Node에 저장해 Tree가 균형잡혀 있는지를 계속 추적한다.

> balanceFactor = height(leftSubTree) − height(rightSubTree)

![](http://interactivepython.org/runestone/static/pythonds/_images/unbalanced.png)

AVL Tree가 Balanced한 상태로 유지되기 위해서는 Balance Factor가 -1, 0, 1 중 하나의 값을 가져야 한다. 위의 그림은 오른쪽으로 더 무거운 Tree의 예이다.

## 6.16. AVL Tree Performance

Big-O performance에 대해 생각해 보기 위해 AVL Tree가 최악으로 한 쪽으로 치우친 경우를 살펴보면 다음과 같다.
![](http://interactivepython.org/runestone/static/pythonds/_images/worstAVL.png)

N<sub>k</sub>를 (위 그림처럼 최악의 상황에서) 깊이가 k일 때 Node의 갯수라 할 때,  
N<sub>0</sub> = 1,  
N<sub>1</sub> = 2,  
N<sub>2</sub> = 4,  
N<sub>3</sub> = 7이며 N<sub>h</sub> = 1 + N<sub>h−1</sub> + N<sub>h−2</sub> 의 관계가 성립한다.

F가 피보나치 수열일 때,  
N<sub>h</sub> = F<sub>h+2</sub> − 1, h≥1

F<sub>i</sub> / F<sub>i−1</sub>가 (1+sqrt(5)) / 2에 수렴하므로,   
h = 1.44logN<sub>h</sub>

깊이가 깊어져도 노드 수가 log하게 증가하므로, 탐색시간이 O(logN)임을 보장할 수 있다

## 6.17. AVL Tree Implementation

Balanced 한 상태를 유지하기 위해, AVL Tree에 Node를 추가할 때에는 생각할 것이 많다.
- Update 시에 Parent의 Balance Factor를 바꾸어야 한다
- Recursive하게 변경사항을 Parent에 반영해야 한다
- Parent Node의 Balance Factor가 0이 되었을 때엔, 그 위의 Ancestor들에는 영향이 없다


In [1]:
def _put(self,key,val,currentNode):
    if key < currentNode.key:
        if currentNode.hasLeftChild():
                self._put(key,val,currentNode.leftChild)
        else:
                currentNode.leftChild = TreeNode(key,val,parent=currentNode)
                self.updateBalance(currentNode.leftChild) # 변경된 부분
    else:
        if currentNode.hasRightChild():
                self._put(key,val,currentNode.rightChild)
        else:
                currentNode.rightChild = TreeNode(key,val,parent=currentNode)
                self.updateBalance(currentNode.rightChild) # 변경된 부분

def updateBalance(self,node):
    if node.balanceFactor > 1 or node.balanceFactor < -1:
        self.rebalance(node) # Tree를 재구성하여 좌우균형을 맞춤
        return
    
    if node.parent != None:
        if node.isLeftChild():
                node.parent.balanceFactor += 1
        elif node.isRightChild():
                node.parent.balanceFactor -= 1

        if node.parent.balanceFactor != 0: # 위의 Ancestor들에게 depth 변경을 전파할 필요가 없다
                self.updateBalance(node.parent)

Tree를 Rebalance하는 방법은 더 치우치지 않은 방향으로 Tree를 회전하는 것이다.

#### Left Rotation
![](http://interactivepython.org/runestone/static/pythonds/_images/simpleunbalanced.png)

- Root의 오른쪽 Child를 Root로 만든다
- 원래 Root를 새로운 Root의 Left Child로 만든다
- 새로운 Root에 이미 Left Child가 있다면, 새로운 Left Child의 Right Child로 만든다(새로운 Left Child는 Right Child가 없음이 보장됨)

#### Right Rotation
![](http://interactivepython.org/runestone/static/pythonds/_images/rightrotate1.png)

- Left Rotation의 거울반전



In [2]:
def rotateLeft(self,rotRoot):
    newRoot = rotRoot.rightChild
    rotRoot.rightChild = newRoot.leftChild
    
    if newRoot.leftChild != None:
        newRoot.leftChild.parent = rotRoot
        
    newRoot.parent = rotRoot.parent
    
    if rotRoot.isRoot():
        self.root = newRoot
    else:
        if rotRoot.isLeftChild():
                rotRoot.parent.leftChild = newRoot
        else:
            rotRoot.parent.rightChild = newRoot
            
    newRoot.leftChild = rotRoot
    rotRoot.parent = newRoot
    
    # 이 부분은 아래 Cell에서 설명
    rotRoot.balanceFactor = rotRoot.balanceFactor + 1 - min(newRoot.balanceFactor, 0) 
    newRoot.balanceFactor = newRoot.balanceFactor + 1 + max(rotRoot.balanceFactor, 0)

![](http://interactivepython.org/runestone/static/pythonds/_images/bfderive.png)
newBal(B) = h<sub>A</sub> − h<sub>C</sub>  
oldBal(B) = h<sub>A</sub> − h<sub>D</sub>

h<sub>D</sub>는 1 + max(h<sub>C</sub>, h<sub>E</sub>)이기 때문에 (아직 바뀌기 전이니 왼쪽 그림 참고),  
oldBal(B) = h<sub>A</sub> − (1 + max(h<sub>C</sub>, h<sub>E</sub>))  
newBal(B) − oldBal(B) = h<sub>A</sub> − h<sub>A</sub> + 1 + max(h<sub>C</sub>, h<sub>E</sub>) − h<sub>C</sub>  
newBal(B) = oldBal(B) + 1 + max(h<sub>C</sub> - h<sub>C</sub>, h<sub>E</sub> - h<sub>C</sub>)

h<sub>E</sub> - h<sub>C</sub>는 −oldBal(D)이므로,  
newBal(B) = oldBal(B) + 1 + max(0,−oldBal(D))  
newBal(B) = oldBal(B) + 1 − min(0, oldBal(D))


왼쪽으로 치우쳐졌다고 반드시 Right Rotation을 하고, 오른쪽으로 치우쳐 졌다고 바로 Left Rotation을 하면 안 된다.

![](http://interactivepython.org/runestone/static/pythonds/_images/hardunbalanced.png)
위와 같은 그림이 오른쪽으로 치우쳐 졌다고 Left Rotation을 해버리면

![](http://interactivepython.org/runestone/static/pythonds/_images/badrotate.png)
이렇게 되어버린다

이러한 상황을 방지하기 위해서는
- Left Rotation이 필요한 상황이라면, Right Child의 Balace Factor를 확인한다. BF가 양수면 Right Child를 기준으로 Right Rotation을 하고, 음수라면 Left Rotation을 한다
- Right Rotation이 필요한 상황이라면, Left Child의 Balace Factor를 확인한다. BF가 양수면 Left Child를 기준으로 Right Rotation을 하고, 음수라면 Left Rotation을 한다

위를 그림으로 나타내면 다음과 같다
![](http://interactivepython.org/runestone/static/pythonds/_images/rotatelr.png)

Tree가 항상 Balanced인 상태 이므로, 이제 get은 O(log<sub>2</sub>n)이며, put의 경우에도 Ancestor들의 balance factor들을 변경하는 것이 최대 log<sub>2</sub>n번 일어나며, rebalancing은 O(1)이므로 결국 O(log<sub>2</sub>n)이다.

## 6.18. Summary of Map ADT Implementations

| operation | Sorted List        | Hash Table | Binary Search Tree | AVL Tree           |
| --------- | ------------------ | ---------- | ------------------ | ------------------ |
| put       | O(n)               | O(1)       | O(n)               | O(log<sub2</sub>n) |
| get       | O(log<sub>2</sub>n) | O(1)       | O(n)               | O(log<sub>2</sub>n) |
| in        | O(log<sub>2</sub>n) | O(1)       | O(n)               | O(log<sub>2</sub>n) |
| del       | O(n)               | O(1)       | O(n)               | O(log<sub2</sub>n) |